In [17]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive

import requests
from urllib.parse import quote
import tkinter as tk
from tkinter import ttk

# Team Casimir Funk Nutritional Content Notebook
In this notebook, we create the our nutritional content functions.

In [20]:
search_query = "Cheddar Cheese"
encoded_query = quote(search_query)

print(encoded_query)

Cheddar%20Cheese


In [53]:
# api_key = 'DEMO_KEY'
# query = 'Cheddar Cheese'
# page_size = 10
# page_number = 1

# url = f'https://api.nal.usda.gov/fdc/v1/foods/search?api_key={api_key}&query={query}&pageSize={page_size}&pageNumber={page_number}'

# response = requests.get(url)

# if response.status_code == 200:
#     data = response.json()
# else:
#     print(f"Error: {response.status_code}")

In [49]:
# def create_scrollable_table(df, master):
#     tree = ttk.Treeview(master)

#     # Add columns to the treeview
#     tree["columns"] = list(df.columns)
#     for col in df.columns:
#         tree.column(col, anchor="w")
#         tree.heading(col, text=col, anchor="w")

#     # Add data to the treeview
#     for i, row in df.iterrows():
#         tree.insert("", i, values=list(row))

#     # Add a scrollbar
#     scrollbar = ttk.Scrollbar(master, orient="vertical", command=tree.yview)
#     tree.configure(yscroll=scrollbar.set)

#     # Pack everything
#     tree.pack(side="left", fill="both", expand=True)
#     scrollbar.pack(side="right", fill="y")

# # Example usage:
# # Assuming you have a DataFrame called 'df'
# root = tk.Tk()
# root.title("Scrollable Table Example")

# # Assuming df is your DataFrame
# # You can replace df with your actual DataFrame
# df = pd.DataFrame({
#     'Name': ['John', 'Alice', 'Bob'],
#     'Age': [25, 30, 22],
#     'City': ['New York', 'San Francisco', 'Los Angeles']
# })

# create_scrollable_table(df, root)
# root.mainloop()


In [55]:
dummy = pd.DataFrame(data['foods'])